In [3]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

In [126]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [127]:
def parse_passport(input: str) -> dict:
    return dict(re.findall(r'([a-z]{3}):([^\s]+)', input))

In [128]:
string = """ecl:blu
eyr:2027 pid:451039029
hcl:#6b5442"""

In [129]:
assert parse_passport(string) == {'ecl': 'blu', 'eyr': '2027', 'pid': '451039029', 'hcl': '#6b5442'}

Part I  

In [130]:
required_fields = set(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])

In [131]:
parsed_data = read_data("input.txt", parser=parse_passport, sep="\n\n")

def validate_passport(parsed_passport: dict) -> boolean:
    return required_fields.issubset(parsed_passport.keys())

sum([validate_passport(entry) for entry in parsed_data])

210

Part II

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.


In [132]:
field_validator = dict(
    byr=lambda v: 1920 <= int(v) <= 2002,
    iyr=lambda v: 2010 <= int(v) <= 2020,
    eyr=lambda v: 2020 <= int(v) <= 2030,
    hgt=lambda v: ((v.endswith('cm') and 150 <= int(v[:-2]) <= 193) or (v.endswith('in') and 59 <= int(v[:-2]) <= 76)),
    hcl=lambda v: re.match(r'^#[0-9a-f]{6}$', v),
    ecl=lambda v: re.match(r'^(amb|blu|brn|gry|grn|hzl|oth)$', v),
    pid=lambda v: re.match(r'^[0-9]{9}$', v),
)


In [156]:
def validate_passport_fields(parsed_passport: dict):
    return all([field_validator[key](parsed_passport[key]) for key in required_fields])

Testing

In [137]:
invalid_strings = """eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
"""

valid_strings = """pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
"""

In [157]:
parsed_passports = read_data(invalid_strings, parser=parse_passport, sep="\n\n", testing=True)
[validate_passport(passport) and validate_passport_fields(passport) for passport in parsed_passports]

[False, False, False, False]

In [158]:
parsed_passports = read_data(valid_strings, parser=parse_passport, sep="\n\n", testing=True)
[validate_passport(passport) and validate_passport_fields(passport) for passport in parsed_passports]

[True, True, True, True]

Actual

In [160]:
sum([validate_passport(entry) and validate_passport_fields(entry) for entry in parsed_data])

131